# Text-to-Image Diffusion (COCO Captions)

Ниже собран ноутбук, который за один запуск проходит через подготовку данных, обучение простого диффузионного U-Net и генерацию изображений по текстовому промпту.

## Датасет: картинки + подписи
Описание: загружаем COCO Captions с Hugging Face, оставляем только картинки и текст подписи. Это обеспечивает пары image-caption, необходимые для условной генерации.

In [ ]:
# Базовые зависимости
import os
from pathlib import Path
import random
import math

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.utils import make_grid
from PIL import Image

from datasets import load_dataset
from transformers import CLIPTokenizer, CLIPTextModel

# Директория для кэша/данных
DATA_DIR = Path('data')
DATA_DIR.mkdir(exist_ok=True)

# Конфигурация
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [ ]:
# Загружаем COCO captions и оставляем только нужные столбцы
raw_dataset = load_dataset('jxie/coco_captions')
columns_to_remove = [c for c in raw_dataset['train'].column_names if c not in ['image', 'captions']]
dataset = raw_dataset.remove_columns(columns_to_remove)
print(dataset)

## Препроцессинг текста и картинок
Описание: приводим изображения к размеру 256×256 и нормализуем в тензоры. Текст опускаем в lower, токенизируем CLIP-токенизатором, создаём input_ids и attention_mask. На выходе готовый Dataset/DataLoader.

In [ ]:
# Трансформации для изображений
to_tensor = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
])

# Токенизатор
TOKENIZER_NAME = 'openai/clip-vit-base-patch32'
tokenizer = CLIPTokenizer.from_pretrained(TOKENIZER_NAME)

class CocoTextImageDataset(Dataset):
    def __init__(self, hf_dataset, tokenizer, transform, max_length=64):
        self.ds = hf_dataset
        self.tokenizer = tokenizer
        self.transform = transform
        self.max_length = max_length

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        item = self.ds[idx]
        image = item['image']
        if not isinstance(image, Image.Image):
            image = Image.open(image).convert('RGB')
        image = self.transform(image)
        caption = item['captions'][0].lower()
        tokens = self.tokenizer(
            caption,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'image': image,
            'input_ids': tokens.input_ids.squeeze(0),
            'attention_mask': tokens.attention_mask.squeeze(0),
        }

# Небольшой сэмпл для демонстрации
train_split = dataset['train'].select(range(200))
train_dataset = CocoTextImageDataset(train_split, tokenizer, to_tensor)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)

batch = next(iter(train_loader))
print(batch['image'].shape, batch['input_ids'].shape, batch['attention_mask'].shape)

## Простая модель (U-Net + текстовый энкодер)
Описание: реализуем упрощённый диффузионный U-Net, который предсказывает шум. Текстовые эмбеддинги получаем CLIPTextModel и добавляем к временным эмбеддингам U-Net через конкатенацию.

In [ ]:
# Диффузионный график (beta schedule) и вспомогательные функции
class DiffusionSchedule:
    def __init__(self, timesteps=200, beta_start=1e-4, beta_end=0.02):
        self.timesteps = timesteps
        self.beta = torch.linspace(beta_start, beta_end, timesteps)
        self.alpha = 1.0 - self.beta
        self.alpha_cumprod = torch.cumprod(self.alpha, dim=0)

    def add_noise(self, x0, noise, t):
        sqrt_alpha = torch.sqrt(self.alpha_cumprod[t])[:, None, None, None]
        sqrt_one_minus_alpha = torch.sqrt(1 - self.alpha_cumprod[t])[:, None, None, None]
        return sqrt_alpha * x0 + sqrt_one_minus_alpha * noise

schedule = DiffusionSchedule(timesteps=200)

# Вспомогательные модули
class SinusoidalPositionEmbeddings(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, timesteps):
        device = timesteps.device
        half_dim = self.dim // 2
        embeddings = math.log(10000) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = timesteps[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
        return embeddings

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, cond_dim):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.time_proj = nn.Linear(cond_dim, out_channels)
        self.act = nn.SiLU()
        self.res_conv = nn.Conv2d(in_channels, out_channels, 1) if in_channels != out_channels else nn.Identity()

    def forward(self, x, cond):
        h = self.conv1(x)
        h = h + self.time_proj(cond)[:, :, None, None]
        h = self.act(h)
        h = self.conv2(h)
        return self.act(h + self.res_conv(x))

class SimpleUNet(nn.Module):
    def __init__(self, base_channels=64, cond_dim=768, time_dim=128):
        super().__init__()
        self.time_mlp = nn.Sequential(
            SinusoidalPositionEmbeddings(time_dim),
            nn.Linear(time_dim, time_dim),
            nn.SiLU(),
            nn.Linear(time_dim, time_dim)
        )
        self.text_proj = nn.Linear(cond_dim, time_dim)

        self.down1 = ResidualBlock(3, base_channels, time_dim)
        self.down2 = ResidualBlock(base_channels, base_channels * 2, time_dim)
        self.pool = nn.AvgPool2d(2)

        self.mid = ResidualBlock(base_channels * 2, base_channels * 2, time_dim)

        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)
        self.up1 = ResidualBlock(base_channels * 2, base_channels, time_dim)
        self.out = nn.Conv2d(base_channels, 3, 1)

    def forward(self, x, t, text_emb):
        t_emb = self.time_mlp(t)
        cond = t_emb + self.text_proj(text_emb)

        d1 = self.down1(x, cond)
        d2 = self.down2(self.pool(d1), cond)
        mid = self.mid(d2, cond)
        u1 = self.up(mid)
        u1 = self.up1(u1 + d1, cond)
        return self.out(u1)

# Текстовый энкодер CLIP
text_encoder = CLIPTextModel.from_pretrained(TOKENIZER_NAME).to(device)
model = SimpleUNet().to(device)

## Тренировка (loss, цикл обучения)
Описание: добавляем шум к реальным изображениям, U-Net предсказывает шум. Используем MSE loss между предсказанным и истинным шумом. Цикл обучения демонстрирует несколько шагов для примера.

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

@torch.no_grad
def encode_text(input_ids, attention_mask):
    enc = text_encoder(input_ids=input_ids, attention_mask=attention_mask)
    # Используем CLS-токен (первый скрытый слой) как эмбеддинг
    return enc.last_hidden_state[:, 0]

num_steps = 5  # демо-значение, при реальном обучении увеличить
model.train()
for step in range(num_steps):
    for batch in train_loader:
        imgs = batch['image'].to(device)
        ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        t = torch.randint(0, schedule.timesteps, (imgs.size(0),), device=device)
        noise = torch.randn_like(imgs)
        noisy = schedule.add_noise(imgs, noise, t)
        text_emb = encode_text(ids, mask)
        pred = model(noisy, t, text_emb)
        loss = nn.functional.mse_loss(pred, noise)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Step {step+1}/{num_steps} - loss: {loss.item():.4f}")

## Генерация картинок по промпту
Описание: начиная с случайного шума, выполняем обратный диффузионный процесс, постепенно убирая шум с учётом текстового эмбеддинга. В конце визуализируем результат.

In [ ]:
@torch.no_grad
def p_sample(model, x, t, text_emb):
    beta_t = schedule.beta[t][:, None, None, None].to(device)
    alpha_t = schedule.alpha[t][:, None, None, None].to(device)
    alpha_bar = schedule.alpha_cumprod[t][:, None, None, None].to(device)

    pred_noise = model(x, t, text_emb)
    mean = (1 / torch.sqrt(alpha_t)) * (x - beta_t / torch.sqrt(1 - alpha_bar) * pred_noise)
    noise = torch.randn_like(x) if (t > 0).all() else torch.zeros_like(x)
    return mean + torch.sqrt(beta_t) * noise

@torch.no_grad
def generate(prompt, steps=50):
    model.eval()
    tokens = tokenizer(prompt, return_tensors='pt', padding='max_length', truncation=True, max_length=64)
    text_emb = encode_text(tokens.input_ids.to(device), tokens.attention_mask.to(device))
    x = torch.randn(1, 3, 256, 256, device=device)
    for i in reversed(range(steps)):
        t = torch.full((1,), i, device=device, dtype=torch.long)
        x = p_sample(model, x, t, text_emb)
    x = (x.clamp(-1, 1) + 1) / 2
    return x

sample = generate("a cat sitting on a sofa", steps=20)
print('Generated sample shape:', sample.shape)

## Итоги
Ноутбук загружает COCO captions, подготавливает изображения и текст, обучает условный U-Net предсказывать шум и умеет генерировать изображение по промпту.